# <font color='darkblue'> THE NETWORK OF BEATLES' SONGS

## <font color='darkblue'>Analysis of the network of Beatles'Songs: Getting into their mind


In [2]:
#We import all the necessary packages we used for this project

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import ast
import operator
import powerlaw
from fa2 import ForceAtlas2
import re
from urllib.request import urlopen
import urllib.request as urllib2
import os

###  <font color='darkblue'>Part 1a: Extraction of information from Beatles Official Page.

In [1]:
#We have create a function to obtain the 'HTLM' of Beatles Official Page. 
def extract_html(url):
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    return html

In [6]:
#We obtain all the list of songs and we stored it in the Variable Songs_Titles
pattern1= 'hreflang="en">(.*?)</a>'
songs_titles=[]
for i in range(0,9): 
    url = 'https://www.thebeatles.com/songs?page='+str(i)
    html=extract_html(url)
    a=re.findall(pattern1,html)
    for el in a:
        el=el.replace('&#039;', '\'')
        songs_titles.append(el)

print('The official Beatles page contains information of a number of' , len(songs_titles), 'songs.')

The official Beatles page contains information of a number of 301 songs.


In [ ]:
remove_list=[' a ',' in ',' the ',' of ',' to ',' is ',' at ',' for ',' that ',' by ',' as ',' from ',' into ',
             ' on ',' with ',' off ',' this ',' up ',' like ']
remove_list_start=['a-','in-','the-','of-','to-','is-','at-','for-','that-','by-','as-','from-','like-','this-',
                  'with-']
remove_list_end=['-by','-to','-on','-is','-that','-before']
remove_list2=['\'','.','!','(',')',',','/',':']

pattern6='<div class="col-md-6 middle-content border-left border-right"><p>'
pattern7='<figure class="wp-block-table table-expander table table-imported">'
for title in songs_titles:
    try:
        title=title.lower()
        for el in remove_list2: # delete special characters
            title=title.replace(el,'')
        for el in remove_list: # replace single words with a space
            title=title.replace(el,' ')
        title=title.replace(' ','-')
        title=title.replace('--','-')
        # delete words from the start
        for i in range(0,6):
            if title[:i] in remove_list_start:
                title=title[i:]
        # delete words end
        for i in range(0,8):
            if title[-i:] in remove_list_end:
                title=title[:-i]
        if title[0]=='-':
            title=title[1:]
        url='https://www.thebeatles.com/'
        query=extract_html(url+title)
        # write the extracted text in a .txt file
        file=open('raw_songs/'+title+'.txt','w+')
        file.write(query)
        file.close()
    except:
        print(title)

Hi ha a vegades que et pila la query del url tot i que la pàgina que correspon a la cançó té un afegit al final de -0 o -1, com el cas de la cançó que peta que és words-love-0

In [ ]:
pattern6='<div class="col-md-6 middle-content border-left border-right"><p>'
pattern7='<figure class="wp-block-table table-expander table table-imported">'
remove_list=['<br />','\n','</p>','<p>']

def extract_lyrics(song_path):
    song=open(song_path).read()
    idx_init = re.search(pattern6,song).end()
    idx_final = re.search(pattern7,song).start()
    lyrics=song[idx_init:idx_final]
    for el in remove_list: # delete special characters
        lyrics=lyrics.replace(el,' ')
    return lyrics

extract_lyrics('raw_songs/ob-la-di-ob-la-da.txt')

In [ ]:
extract_lyrics('raw_songs/yesterday.txt')

In [ ]:
txt_files = os.listdir('raw_songs')
songs_no_lyrics=[]
for el in txt_files:
    el=el[:-4]
    song_path='raw_songs/'+el+'.txt'
    try:
        lyrics=extract_lyrics(song_path)
        file=open('lyrics_songs/'+el+'.txt','w+')
        file.write(lyrics)
        file.close()
    except:
        songs_no_lyrics.append(el)
        print(el, end='\t')

In [ ]:
lyrics_files = os.listdir('lyrics_songs')
for el in songs_no_lyrics:
    el=el[:-4]
    url='https://www.thebeatles.com/'
    # check if the song has lyrics or not
    for i in range(0,4):
        try:
            query=extract_html(url+el+'-'+str(i))
            idx_init = re.search(pattern6,query).end()
            idx_final = re.search(pattern7,query).start()
            # rewrite the extracted text in a .txt file
            file=open('raw_songs/'+el+'.txt','w+')
            file.write(query)
            file.close()
            print(el, i)
            break
        except:
            pass

In [ ]:
txt_files = os.listdir('raw_songs')
songs_no_lyrics=[]
for el in txt_files:
    song_path='raw_songs/'+el
    try:
        lyrics=extract_lyrics(song_path)
        file=open('lyrics_songs/'+el+'.txt','w+')
        file.write(lyrics)
        file.close()
    except:
        songs_no_lyrics.append(el)
len(songs_no_lyrics)

### Feature: Clean lyrics files

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.probability import FreqDist

stop_words = list(set(stopwords.words('english')))
stop_words = [el.replace('\'','') for el in stop_words]
stop_words.append('im')


def clean_lyrics(file_path):
    data=open(file_path).read()
    # import WordPunctTokenizer() method from nltk
    # Create a reference variable for Class WordPunctTokenizer
    tk = WordPunctTokenizer()
    # define punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # remove punctuation from the string
    no_punct = ""
    for char in data:
        if char not in punctuations:
            no_punct = no_punct + char
    # Remove all the special characters as \n and single = left
    char=['\n','=']
    for el in char:
        raw=re.sub(el,'',no_punct)
    # remove stop words
    token_txt = tk.tokenize(raw.lower()) # set to lower case
    token_txt = tk.tokenize(raw.lower()) # set to lower case
    token_final = [x for x in token_txt if x not in stop_words and len(x)>2]
    return token_final

In [ ]:
lyrics_files=[el[:-4] for el in os.listdir('lyrics_songs')]
lyrics_files.remove('.DS_S') # remove this element that is introduced when using os.listdir
len(lyrics_files)

## Create network

In [ ]:
d_top5={}
for file in lyrics_files:
    path='lyrics_songs/'+file+'.txt'
    clean=clean_lyrics(path)
    
    top5=[]
    for el in FreqDist(clean).most_common(5):
        top5.append(el[0])
    d_top5[file]=top5

In [ ]:
hyperlinks={} #dictionary to store links 
for file1 in lyrics_files:
    eq_files=[]
    for file2 in lyrics_files:
        if file1!=file2:
            a=d_top5[file1]
            b=d_top5[file2]
            # compute if there are equal words in both lists
            eq=0
            for el in a:
                if el in b:
                    eq+=1
            if eq!=0:
                eq_files.append(file2)
    hyperlinks[file1]=eq_files

In [ ]:
import networkx as nx
#We build the Directed Network
Beatles_nw= nx.DiGraph()
for file in lyrics_files:
    #We add the nodes and attributes to the the network
    Beatles_nw.add_node(file)


#We add the hyperlinks to the the network
for el in lyrics_files:
    links= hyperlinks[el]
    for a in links:
        u= el
        v= a
        Beatles_nw.add_edge(u,v)

In [ ]:
nx.draw(Beatles_nw)

### Extract GCC

In [ ]:
Gs = list(Beatles_nw.subgraph(c).copy() for c in nx.weakly_connected_components(Beatles_nw))
biggest = 0
GCC_index = 0
for index,graph in enumerate(Gs):
    if len(graph.nodes) > biggest:
        biggest = len(graph.nodes)
        GCC_index = index
GCC = Gs[GCC_index]

In [ ]:
options = {
    'node_size': 40,
    'width': 0.2,
}
nx.draw(GCC,**options)

#### Convert to undirected network

In [ ]:
un_Beatles_nw= GCC.to_undirected()

options = {
    'node_size': 40,
    'width': 0.2,
}
nx.draw(un_Beatles_nw,**options)

Still needs to be define a criteria to link two songs. Right now the criteria is that if between the most 5 common words one is present in both songs, these songs are linked.

In [ ]:
Sorted_degree=dict(sorted(un_Beatles_nw.degree, key=lambda x: x[1], reverse=True))
Sorted_degree

In [ ]:
d_top5['dont-pass-me']

In [ ]:
d_top5['love-me-do']

In [ ]:
d_top5['i-feel-fine']

In [ ]:
d_top5['yellow-submarine']

#### Plot degree distribution

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#FROM UNDIRECTED TO DIRECTED GRAPH:
un_Beatles_nw= GCC.to_undirected()
all_degrees=[]

#Obtenition of degree 
for deg in un_Beatles_nw.degree():
    all_degrees.append(deg[1])
    
#Zoom in
v = np.arange(int(min(all_degrees)),max(Sorted_degree.values()))
count,bins=np.histogram(all_degrees,bins=v)
plt.bar(bins[:-1], count)
plt.title('Zoom in: Degree distribution of the undirected graph',fontsize=13)
plt.xlabel('Degree',fontsize=12)
plt.ylabel('Counts',fontsize=12)
plt.show() 

## Find communities

In [ ]:
import community.community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

# compute the best partition
partition = community.community_louvain.best_partition(un_Beatles_nw)

# draw the graph
pos = nx.spring_layout(un_Beatles_nw)
# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(un_Beatles_nw, pos, partition.keys(), node_size=20,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(un_Beatles_nw, pos, alpha=0.5)
plt.show()

In [ ]:
print('Number of communities found: ', len(np.unique(list(partition.values()))))

In [ ]:
print('Modularity:', np.round(community.community_louvain.modularity(partition,un_Beatles_nw),4))

In [ ]:
# we compute the number of characters in every community in order to find the top 5
communities_d=dict.fromkeys(list(range(0,len(np.unique(list(partition.values()))))))
for i in range(0,len(np.unique(list(partition.values())))):
    l=[]
    for el in partition:
        if partition[el]==i:
            l.append(el)
    communities_d[i]=l
    
size_communities=[len(x) for x in communities_d.values()]

size_communities

Extract the most common words for each community

In [ ]:
communities_d[4]

Most common words in each community (counting the time each word appears)

In [ ]:
for idx in communities_d:
    community_words=[]
    for el in communities_d[idx]:
        path='lyrics_songs/'+el+'.txt'
        clean=clean_lyrics(path)
        for word in clean:
            community_words.append(word)
    print(FreqDist(community_words).most_common(5))

Most common words in each community (counting the number of song a word appear)

In [ ]:
for idx in communities_d:
    community_words=[]
    for el in communities_d[idx]:
        path='lyrics_songs/'+el+'.txt'
        clean=clean_lyrics(path)
        unique=list(np.unique(clean))
        for word in unique:
            community_words.append(word)
    print('size:' ,size_communities[idx])
    print(FreqDist(community_words).most_common(5))

### Feature: most common words of a given community (times a song contains a word)

In [ ]:
def community_words(community):
    community_words=[]
    for el in community:
        path='lyrics_songs/'+el+'.txt'
        clean=clean_lyrics(path)
        unique=list(np.unique(clean))
        for word in unique:
            community_words.append(word)
    return FreqDist(community_words).most_common(5)

community_words(communities_d[0])

### Feature: introduce a famous song and find in which community it belongs

In [ ]:
def song_community(song):
    i=0
    for idx in communities_d:
        if song in communities_d[i]:
            return i
        i+=1
        
song_community('let-it-be')

In [ ]:
community_words(communities_d[song_community('let-it-be')])

## VADER sentimental analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
sentenceDic = {}
for name in dialogues:
    sentenceDic[name] = []
    for line in dialogues[name]:
        sentence = re.split(r'[.!?]+ *',line)
        sentenceDic[name].extend(sentence)

In [ ]:
file=open('lyrics_songs/youll-be-mine.txt').read()
file

In [ ]:
sentences=[]
index=[(m.start(0), m.end(0)) for m in re.finditer('  ',file)]
for i in range(0,len(index)-1):
    a=file[index[i][1]:index[i+1][0]]
    sentences.append(a)

In [ ]:
vs['compound']

In [ ]:
def song_Vader(song):
    # extract sentence of each song
    sentences=[]
    index=[(m.start(0), m.end(0)) for m in re.finditer('  ',file)]
    for i in range(0,len(index)-1):
        a=file[index[i][1]:index[i+1][0]]
        sentences.append(a)
    # compute sentiment analysis for sentence
    analyzer = SentimentIntensityAnalyzer()
    compound1=[]
    compound2=[]
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        # take every sentence value (more robust)
        compound1.append(vs['compound'])
        # take just the sentences that are not neutral
        if vs['compound']!=0:
            compound2.append(vs['compound'])
        # print("{:-<65} {}".format(sentence, str(vs)))
    return np.mean(compound1),np.mean(compound2)

song_Vader('youll-be-mine')[0]

In [ ]:
for idx in communities_d:
    compound1=[]
    compound2=[]
    for el in communities_d[idx]:
        print(compound1.append(song_Vader(el)[0]))
        compound2.append(song_Vader(el)[1])
    print(np.mean(compound1))

In [ ]:
sentenceDic = {}
for name in dialogues:
    sentenceDic[name] = []
    for line in dialogues[name]:
        sentence = re.split(r'[.!?]+ *',line)
        sentenceDic[name].extend(sentence)

In [ ]:
def sentiment_score_VADER(sentence):
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

In [ ]:
sentiment_score_VADER('Oh yes, you\'ll be.')

In [ ]:
for idx in communities_d:
    for el in communities_d[idx]:
        